Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/deployment/deploy-multi-model/multi-model-register-and-deploy.png)

# Deploy Multiple Models as Webservice

This example shows how to deploy a Webservice with multiple models in step-by-step fashion:

 1. Register Models
 2. Deploy Models as Webservice

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration](../../../configuration.ipynb) Notebook first if you haven't.

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.22.0


## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

fmoz-workspace
ml
westus2
421b563f-a977-42aa-8934-f41ca5664b73


## Register Models

In this example, we will be using and registering two models. 

First we will train two simple models on the [diabetes dataset](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset) included with scikit-learn, serializing them to files in the current directory.

In [3]:
import joblib
import sklearn

from sklearn.datasets import load_diabetes
from sklearn.linear_model import BayesianRidge, Ridge

x, y = load_diabetes(return_X_y=True)

first_model = Ridge().fit(x, y)
second_model = BayesianRidge().fit(x, y)

joblib.dump(first_model, "first_model.pkl")
joblib.dump(second_model, "second_model.pkl")

print("Trained models using scikit-learn {}.".format(sklearn.__version__))

Trained models using scikit-learn 0.22.2.post1.


Now that we have our trained models locally, we will register them as Models with the names `my_first_model` and `my_second_model` in the workspace.

In [4]:
from azureml.core.model import Model

my_model_1 = Model.register(model_path="first_model.pkl",
                            model_name="my_first_model",
                            workspace=ws)

my_model_2 = Model.register(model_path="second_model.pkl",
                            model_name="my_second_model",
                            workspace=ws)

Registering model my_first_model
Registering model my_second_model


## Write the Entry Script
Write the script that will be used to predict on your models

### Model.get_model_path()

To get the paths of your models, use `Model.get_model_path(model_name, version=None, _workspace=None)` method. This method will find the path to a model using the name of the model registered under the workspace.

In this example, we do not use the optional arguments `version` and `_workspace`.

#### Using environment variable AZUREML_MODEL_DIR

In other [examples](../deploy-to-cloud/score.py) with a single model deployment, we use the environment variable `AZUREML_MODEL_DIR` and model file name to get the model path. 

For single model deployments, this environment variable is the path to the model folder (`./azureml-models/$MODEL_NAME/$VERSION`). When we deploy multiple models, the environment variable is set to the folder containing all models (./azureml-models).

If you're using multiple models and you know the versions of the models you deploy, you can use this method to get the model path:

```python
# Construct the model path using the registered model name, version, and model file name
model_1_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'my_first_model', '1', 'first_model.pkl')
```

In [5]:
%%writefile score.py
import joblib
import json
import numpy as np

from azureml.core.model import Model

def init():
    global model_1, model_2
    # Here "my_first_model" is the name of the model registered under the workspace.
    # This call will return the path to the .pkl file on the local disk.
    model_1_path = Model.get_model_path(model_name='my_first_model')
    model_2_path = Model.get_model_path(model_name='my_second_model')
    
    # Deserialize the model files back into scikit-learn models.
    model_1 = joblib.load(model_1_path)
    model_2 = joblib.load(model_2_path)

# Note you can pass in multiple rows for scoring.
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = np.array(data)
        
        # Call predict() on each model
        result_1 = model_1.predict(data)
        result_2 = model_2.predict(data)

        # You can return any JSON-serializable value.
        return {"prediction1": result_1.tolist(), "prediction2": result_2.tolist()}
    except Exception as e:
        result = str(e)
        return result

Overwriting score.py


## Create Environment

You can now create and/or use an Environment object when deploying a Webservice. The Environment can have been previously registered with your Workspace, or it will be registered with it as a part of the Webservice deployment. Please note that your environment must include azureml-defaults with verion >= 1.0.45 as a pip dependency, because it contains the functionality needed to host the model as a web service.

More information can be found in our [using environments notebook](../training/using-environments/using-environments.ipynb).

In [6]:
from azureml.core import Environment

env = Environment("deploytocloudenv")
env.python.conda_dependencies.add_pip_package("joblib")
env.python.conda_dependencies.add_pip_package("numpy")
env.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

## Create Inference Configuration

There is now support for a source directory, you can upload an entire folder from your local machine as dependencies for the Webservice.
Note: in that case, environments's entry_script and file_path are relative paths to the source_directory path; myenv.docker.base_dockerfile is a string containing extra docker steps or contents of the docker file.

Sample code for using a source directory:

```python
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

myenv = Environment.from_conda_specification(name='myenv', file_path='env/myenv.yml')

# explicitly set base_image to None when setting base_dockerfile
myenv.docker.base_image = None
# add extra docker commends to execute
myenv.docker.base_dockerfile = "FROM ubuntu\n RUN echo \"hello\""

inference_config = InferenceConfig(source_directory="C:/abc",
                                   entry_script="x/y/score.py",
                                   environment=myenv)
```

 - file_path: input parameter to Environment constructor. Manages conda and python package dependencies.
 - env.docker.base_dockerfile: any extra steps you want to inject into docker file
 - source_directory: holds source path as string, this entire folder gets added in image so its really easy to access any files within this folder or subfolder
 - entry_script: contains logic specific to initializing your model and running predictions

In [7]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py", environment=env)

### Deploy Model as Webservice on Azure Container Instance

Note that the service creation can take few minutes.

In [8]:
from azureml.core.webservice import AciWebservice

aci_service_name = "aciservice-multimodel"

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(ws, aci_service_name, [my_model_1, my_model_2], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)

print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....................................................

#### Test web service

In [ ]:
import json

test_sample = json.dumps({'data': x[0:2].tolist()})

prediction = service.run(test_sample)

print(prediction)

#### Delete ACI to clean up

In [ ]:
service.delete()